<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Formula" data-toc-modified-id="Formula-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Formula</a></span><ul class="toc-item"><li><span><a href="#Debug-Params-(Last-Month)" data-toc-modified-id="Debug-Params-(Last-Month)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Debug Params (Last Month)</a></span></li></ul></li><li><span><a href="#PLOT-REGISTRATION" data-toc-modified-id="PLOT-REGISTRATION-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PLOT REGISTRATION</a></span><ul class="toc-item"><li><span><a href="#Maps" data-toc-modified-id="Maps-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Maps</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Summary</a></span></li></ul></li><li><span><a href="#Weather" data-toc-modified-id="Weather-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Weather</a></span></li><li><span><a href="#SOIL" data-toc-modified-id="SOIL-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SOIL</a></span></li><li><span><a href="#Plant-Health" data-toc-modified-id="Plant-Health-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plant Health</a></span><ul class="toc-item"><li><span><a href="#General-Condition" data-toc-modified-id="General-Condition-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>General Condition</a></span></li><li><span><a href="#Plant-Nodes" data-toc-modified-id="Plant-Nodes-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Plant Nodes</a></span></li><li><span><a href="#Plant-Leaf" data-toc-modified-id="Plant-Leaf-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Plant Leaf</a></span></li><li><span><a href="#Plant-Branch" data-toc-modified-id="Plant-Branch-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Plant Branch</a></span></li><li><span><a href="#Other-Information" data-toc-modified-id="Other-Information-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Other Information</a></span></li></ul></li><li><span><a href="#SEND-EMAIL" data-toc-modified-id="SEND-EMAIL-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>SEND EMAIL</a></span></li></ul></div>

In [1]:
from datetime import datetime,timedelta
from pytz import utc, timezone
import time
from Akvo import Flow
from FlowHandler import FlowHandler
import pandas as pd
import json
import os

## Formula

**Nitrate**

- N = (x + 0.613) * DF / 2.7763
- DF = (1 + (SM * 1.3)) * 5
- SM is Soil Moisture

--------------

Phospate N/A not Zero
Mean only recorded values

In [13]:
import smtplib
import sys
from email import encoders
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [3]:
instanceURI = 'spiceup'
requestURI = 'https://api.akvo.org/flow/orgs/' + instanceURI
EMAIL_RECEPIENTS = ['mail@dedenbangkit.com','ima@akvo.org','joy@akvo.org']
EMAIL_SENDER = os.environ['G4AW_EMAIL']
EMAIL_PASSWORD = os.environ['G4AW_PWD']

In [4]:
apiData = Flow.getResponse(requestURI + '/surveys/227030237')
#apiData = Flow.getResponse(requestURI + '/surveys/24210001')
forms = apiData.get('forms')
RegistrationForm = apiData['registrationFormId']
start_time = time.time()
date_format = '%Y-%m-%dT%H:%M:%SZ'

In [5]:
today_date = datetime.today().date()
week_ago = today_date - timedelta(days=30)

In [6]:
def formatDate(dt):
    return dt.strftime("%d/%m/%Y")

In [7]:
def checkTime(x):
    total_time = x - start_time
    spent = time.strftime("%H:%M:%S", time.gmtime(total_time))
    return spent

In [8]:
questions = lambda x : [{'id':a['id'],'name':a['name'],'questions':details(a['questions'])} for a in x]
details = lambda x : [{'id':a['id'],'name':a['name'].replace(' ','_'),'type':a['type']} for a in x]

In [9]:
registrationFormId = apiData.get('registrationFormId')

In [10]:
def getAll(url):
    data = Flow.getResponse(url)
    formInstances = data.get('formInstances')
    for dataPoint in formInstances:
        dataPoints.append(dataPoint)
    try:
        print(checkTime(time.time()) + ':: GET DATA FROM[' + url + ']')
        url = data.get('nextPageUrl')
        getAll(url)
    except:
        #print(checkTime(time.time()) + ':: DOWNLOAD COMPLETE')
        return "done"

In [11]:
allResponses = {}
for form in forms:
    questionGroups = questions(form['questionGroups'])
    metas = pd.DataFrame(questionGroups)
    formURI = form['formInstancesUrl']
    allGroups = {}
    for index, questionGroup in enumerate(questionGroups):
        dataPoints = []
        groupID = questionGroup['id']
        metadata = metas['questions'][index]
        getAll(formURI)
        output = pd.DataFrame(dataPoints)
        #print(checkTime(time.time()) + ':: TRANSFORMING')
        for qst in metadata:
            qName = qst['name'].replace('_',' ')
            qId = str(qst['id'])
            qType = qst['type']
            try:
                output[qName] = output['responses'].apply(lambda x: FlowHandler(x[groupID],qId,qType))
                if qType == 'GEO':
                    output[qName+'_lat'] = output[qName].apply(lambda x: x[0] if x is not None else x)
                    output[qName+'_long'] = output[qName].apply(lambda x: x[1] if x is not None else x)
                    output = output.drop([qName], axis=1)
            except:
                pass
        try:
            output = output.drop(['responses'], axis=1)
        except:
            pass
        allGroups.update({questionGroup['name']:output.to_dict('records')})
    allResponses.update({form['name']:allGroups})

00:00:03:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=237020121]
00:00:06:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=237020121&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGMO-qm8MGAAgAA]
00:00:12:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331]
00:00:17:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGOGAlmYMGAAgAA]
00:00:21:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGMLi_WYMGAAgAA]
00:00:25:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGKXpgmgMGAAg

00:03:27:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGPbpqHcMGAAgAA]
00:03:30:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGIbhv3kMGAAgAA]
00:03:35:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331]
00:03:39:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGOGAlmYMGAAgAA]
00:03:42:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGMLi_WYMGAAgAA]
00:03:46:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=2470503

00:06:18:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGPbpqHcMGAAgAA]
00:06:21:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGIbhv3kMGAAgAA]
00:06:27:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331]
00:06:30:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGOGAlmYMGAAgAA]
00:06:34:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331&cursor=Ci8SKWoOZX5ha3ZvZmxvdy0xNzdyFwsSDlN1cnZleUluc3RhbmNlGMLi_WYMGAAgAA]
00:06:37:: GET DATA FROM[https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=2470503

In [15]:
pd.DataFrame(allResponses).unstack().to_frame().dropna().reset_index()

level_0             level_1  \
0    01. Plot Registration  Plant Registration   
1     04. Plant Monitoring   General Condition   
2     04. Plant Monitoring         Plant Nodes   
3     04. Plant Monitoring          Plant Leaf   
4     04. Plant Monitoring        Plant Branch   
5     04. Plant Monitoring   Other Information   
6             05. Activity            Activity   
7             05. Activity         Fertilizing   
8             05. Activity           Pesticide   
9             05. Activity            Watering   
10  02. Weather Conditions  Weather Conditions   
11     03. Soil Monitoring     Soil Properties   

                                                    0  
0   [{'deviceIdentifier': 'PT CAN soil', 'dataPoin...  
1   [{'deviceIdentifier': 'CAN', 'dataPointId': '2...  
2   [{'deviceIdentifier': 'CAN', 'dataPointId': '2...  
3   [{'deviceIdentifier': 'CAN', 'dataPointId': '2...  
4   [{'deviceIdentifier': 'CAN', 'dataPointId': '2...  
5   [{'deviceIdentifier': 'CAN', 'dataPointId': '2...  
6                                                  []  
7                                                  []  
8                                                  []  
9                                                  []  
10  [{'deviceIdentifier': 'CAN', 'dataPointId': '2...  
11  [{'deviceIdentifier': 'PT CAN soil', 'dataPoin...

In [16]:
pd.DataFrame(questionGroups)

id             name  \
0  245020170  Soil Properties   

                                           questions  
0  [{'id': '224750001', 'name': 'ID', 'type': 'FR...

In [17]:
qGroup = list(c.keys())
qGroup.sort()

NameError: name 'c' is not defined

In [18]:
allResponses[qGroup[3]]

NameError: name 'qGroup' is not defined

In [19]:
newResponses = {}
newSort = sorted(allResponses.keys())
for ns in newSort:
    newResponses.update({ns:allResponses[ns]})

In [20]:
for g in qGroup:
    forms = [k for k in newResponses[g]]
    for form in forms:
        unordered = pd.DataFrame(newResponses[g][form])
        print(list(unordered))
        try:
            ordered = unordered.sort_values(by='submissionDate',ascending=False).to_dict('records')
            newResponses[g][form] = ordered
        except:
            pass
allResponses = newResponses

NameError: name 'qGroup' is not defined

In [21]:
this_month = datetime.now().strftime('%B')

### Debug Params (Last Month)

In [22]:
#this_month = today_date - timedelta(days=30)

In [23]:
email_data = {}

## PLOT REGISTRATION

In [24]:
plant = allResponses['01. Plot Registration']['Plant Registration'][0]
plot_location = plant['Plot Location'].split('|')
plot_location = [pl.split(':')[0] for pl in plot_location]
plot_location = ', '.join(plot_location)
plot_location

'Kepulauan Bangka Belitung, Kab. Bangka TengahKab. Bangka Tengah, NamangNamang, Desa NamangDesa Namang'

In [25]:
email_data.update({'plot_location':plot_location})

### Maps

In [26]:
plot_latlong = [str(plant['Plot Geopoint_lat']),str(plant['Plot Geopoint_long'])]
plot_latlong = ', '.join(plot_latlong)
plot_map_img = "https://www.mapquestapi.com/staticmap/v4/getmap?size=640,480&type=map&pois=mcenter," + plot_latlong
plot_map_img = plot_map_img +"|&zoom=12&center="+ plot_latlong +"&imagetype=JPEG&traffic=flow&scalebar=false&key=lYrP4vF3Uk5zgTiGGuEzQGwGIVDGuy24"

In [27]:
plot_link = "http://maps.google.com/maps?t=k&q=loc:" + plot_latlong

In [28]:
plot_link

'http://maps.google.com/maps?t=k&q=loc:-2.3428743096598823, 106.18896435946226'

In [29]:
email_data.update({'plot_link':plot_link})

### Summary

In [30]:
plant_submissionDate = plant['submissionDate'].split('T')[0]
plot_submission_summary = 'First demonstration plots have been planted at '
plot_submission_summary =  plot_submission_summary + datetime.strptime(plant_submissionDate, '%Y-%m-%d').strftime('%B, %d')
plant_type = plant['Type of Plantation']
plot_submission_summary = plot_submission_summary + ' with ' + plant_type + ' in ' + plot_location + '.'

In [31]:
plot_submission_summary

'First demonstration plots have been planted at April, 16 with White Pepper in Kepulauan Bangka Belitung, Kab. Bangka TengahKab. Bangka Tengah, NamangNamang, Desa NamangDesa Namang.'

In [32]:
email_data.update({'plot_submission_summary':plot_submission_summary})

## Weather

In [33]:
weather = pd.DataFrame(allResponses['02. Weather Conditions']['Weather Conditions'])
weather['General Weather Conditions'] = weather['General Weather Conditions'].apply(lambda x: x.replace(':',''))
weather['createdAt'] = weather['createdAt'].astype('datetime64[ns]')
weather['month'] = weather['createdAt'].apply(lambda x : x.strftime('%B'))
weather = weather[weather['month'] == this_month]
weather_average_temperature = list(weather.groupby('identifier').mean().reset_index()['Temperature (Celsius)'])[0]
weather_average_temperature = str(int(weather_average_temperature)) + '° Celcious'
weather_average_temperature

'27° Celcious'

In [34]:
email_data.update({'weather_average_temperature':weather_average_temperature})

In [35]:
weather_most_condition = weather.groupby(['General Weather Conditions','createdAt']).size().to_frame('size').reset_index()
weather_most_condition = weather_most_condition.groupby(['General Weather Conditions']).size().to_frame('size').reset_index()
weather_most_condition = weather_most_condition.max()[0]
weather_most_condition = 'Mostly ' + weather_most_condition

In [36]:
weather_most_condition

'Mostly Sunny'

In [37]:
email_data.update({'weather_most_condition':weather_most_condition})

In [38]:
weather_all_conditions = weather[['dataPointId','General Weather Conditions']].groupby(['General Weather Conditions']).count().reset_index()

In [39]:
email_data.update({'weather_all_conditions':weather_all_conditions.to_dict('records')})

## SOIL

In [40]:
soil = pd.DataFrame(allResponses['03. Soil Monitoring']['Soil Properties'])
soil['createdAt'] = soil['createdAt'].astype('datetime64[ns]')
soil['month'] = soil['createdAt'].apply(lambda x : x.strftime('%B'))
soil = soil[soil['month'] == this_month]

In [41]:
def get_soil_result(x):
    if x is not None:
        return float(x['result'][0]['value'])
    else:
        return 0

In [42]:
soil['Soil Moisture'] = soil['Soil Moisture'].apply(get_soil_result)
soil['Soil pH'] = soil['Soil pH'].apply(get_soil_result)
soil['Nitrate'] = soil['Nitrate'].apply(get_soil_result)
soil['P'] = soil['P'].apply(get_soil_result)

In [43]:
soil.groupby('identifier').mean().reset_index()

identifier  surveyalTime  Soil Moisture   Soil pH   Nitrate    P
0  23jy-48c7-2pax     48.666667       2.656111  2.622222  2.388889  0.0

In [44]:
pd.DataFrame(allResponses['03. Soil Monitoring']['Soil Properties'])

deviceIdentifier dataPointId        submissionDate  \
0       PT CAN soil   245520001  2019-10-15T03:15:09Z   
1       PT CAN soil   245520001  2019-10-17T03:05:52Z   
2       PT CAN soil   245520001  2019-10-17T04:06:47Z   
3       PT CAN soil   245520001  2019-05-28T08:41:29Z   
4       PT CAN soil   245520001  2019-08-10T09:47:55Z   
5       PT CAN soil   245520001  2019-05-28T09:28:49Z   
6       PT CAN soil   245520001  2019-09-08T01:49:30Z   
7       PT CAN soil   245520001  2019-05-28T09:17:33Z   
8       PT CAN soil   245520001  2019-10-15T03:31:44Z   
9       PT CAN soil   245520001  2019-10-16T09:27:10Z   
10      PT CAN soil   245520001  2019-10-15T03:36:26Z   
11      PT CAN soil   245520001  2019-10-15T03:21:53Z   
12      PT CAN soil   245520001  2019-05-28T08:59:24Z   
13      PT CAN soil   245520001  2019-10-15T03:26:52Z   
14      PT CAN soil   245520001  2019-10-15T06:14:11Z   
15      PT CAN soil   245520001  2019-10-17T03:56:37Z   
16      PT CAN soil   245520001  2019-10-15T03:39:37Z   
17      PT CAN soil   245520001  2019-10-15T06:10:19Z   
18      PT CAN soil   245520001  2019-10-15T03:01:40Z   
19      PT CAN soil   245520001  2019-10-15T02:45:18Z   
20      PT CAN soil   245520001  2019-10-17T01:17:04Z   
21      PT CAN soil   245520001  2019-10-17T02:13:53Z   
22      PT CAN soil   245520001  2019-10-17T01:43:29Z   
23      PT CAN soil   245520001  2019-10-17T03:49:43Z   

                  modifiedAt         id      identifier  \
0   2019-10-15T06:13:04.252Z  214570003  23jy-48c7-2pax   
1   2019-10-17T03:06:42.904Z  214590002  23jy-48c7-2pax   
2   2019-10-17T04:07:38.378Z  214610003  23jy-48c7-2pax   
3   2019-05-28T08:42:14.826Z  215430004  23jy-48c7-2pax   
4   2019-08-10T09:48:43.460Z  215730001  23jy-48c7-2pax   
5   2019-05-28T09:29:30.242Z  219780003  23jy-48c7-2pax   
6   2019-09-08T02:20:52.685Z  226210056  23jy-48c7-2pax   
7   2019-05-28T09:18:18.356Z  227880002  23jy-48c7-2pax   
8   2019-10-15T03:43:28.756Z  228640006  23jy-48c7-2pax   
9   2019-10-16T09:28:20.631Z  230730001  23jy-48c7-2pax   
10  2019-10-15T03:43:33.798Z  232510002  23jy-48c7-2pax   
11  2019-10-15T06:13:10.489Z  234570004  23jy-48c7-2pax   
12  2019-05-28T09:00:31.557Z  239640004  23jy-48c7-2pax   
13  2019-10-15T03:43:16.761Z  240420008  23jy-48c7-2pax   
14  2019-10-15T06:14:54.968Z  240420012  23jy-48c7-2pax   
15  2019-10-17T03:57:28.261Z  242570004  23jy-48c7-2pax   
16  2019-10-15T03:43:49.181Z  246540006  23jy-48c7-2pax   
17  2019-10-15T06:13:10.553Z  248790006  23jy-48c7-2pax   
18  2019-10-15T03:02:25.539Z  252630004  23jy-48c7-2pax   
19  2019-10-15T02:57:55.805Z  254800008  23jy-48c7-2pax   
20  2019-10-17T01:18:01.874Z  254820001  23jy-48c7-2pax   
21  2019-10-17T02:14:37.670Z  254820002  23jy-48c7-2pax   
22  2019-10-17T01:44:13.264Z  254840001  23jy-48c7-2pax   
23  2019-10-17T03:50:27.288Z  254840002  23jy-48c7-2pax   

                                          displayName     formId  \
0   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
1   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
2   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
3   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
4   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
5   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
6   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
7   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
8   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
9   Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
10  Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
11  Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
12  Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
13  Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
14  Demo Plot Bangka - White Pepper - Kepulauan Ba...  227030238   
15  Demo Plot

In [45]:
Flow.getResponse('https://api.akvo.org/flow/orgs/spiceup/form_instances?survey_id=227030237&form_id=247050331')

{'formInstances': [{'deviceIdentifier': 'CAN',
   'dataPointId': '245520001',
   'submissionDate': '2019-08-27T06:20:58Z',
   'modifiedAt': '2019-08-27T12:46:51.811Z',
   'id': '214050008',
   'responses': {'238950116': [{'217070156': 80.0, '241000338': 'N10'}],
    '222990342': [{'227030242': 6.0, '247060418': 2.0}],
    '235080147': [{'235080148': 17.0}],
    '245010211': [{'240920201': 48.0}],
    '245010212': [{'245010210': {'filename': 'https://akvoflow-177.s3.amazonaws.com/images/8e8a721f-1547-4552-b312-4c287d732ccc.jpg'},
      '247100136': '06'}]},
   'identifier': '23jy-48c7-2pax',
   'displayName': 'Demo Plot Bangka - White Pepper - Kepulauan Bangka Belitung - Kab. Bangka Tengah - Namang - Desa Namang',
   'formId': '247050331',
   'surveyalTime': 29,
   'submitter': 'Indri',
   'createdAt': '2019-08-27T12:46:48.633Z'},
  {'deviceIdentifier': 'CAN',
   'dataPointId': '245520001',
   'submissionDate': '2019-08-27T06:37:35Z',
   'modifiedAt': '2019-08-27T12:47:02.207Z',
   'id'

## Plant Health

In [46]:
planthealth = pd.DataFrame(allResponses['04. Plant Monitoring'])

### General Condition

In [47]:
plant_gc = pd.DataFrame(allResponses['04. Plant Monitoring']['General Condition'])

In [48]:
plant_gc['createdAt'] = plant_gc['createdAt'].astype('datetime64[ns]')
plant_gc['month'] = plant_gc['createdAt'].apply(lambda x : x.strftime('%B'))
plant_gc = plant_gc[plant_gc['month'] == this_month]

In [49]:
plant_height_max = plant_gc.max()
plant_height_min = plant_gc.min()
plant_height_avg = plant_gc.mean()

In [50]:
variables = ['Plant / Vine Condition','Roots / Stem Base']
conditions = plant_gc[variables + ['Plant ID']].groupby(variables).count().reset_index().to_dict('records')
for condition in conditions:
    print(str(condition['Plant ID']) + " plants exhibited " + condition['Roots / Stem Base'].lower() + " roots and " + condition['Plant / Vine Condition'].lower() + " vine condition")

80 plants exhibited whitish roots and vigourous vine condition
27 plants exhibited whitish roots and wilt vine condition


### Plant Nodes

In [51]:
plant_node = pd.DataFrame(allResponses['04. Plant Monitoring']['Plant Nodes'])

In [52]:
plant_node['createdAt'] = plant_node['createdAt'].astype('datetime64[ns]')
plant_node['month'] = plant_node['createdAt'].apply(lambda x : x.strftime('%B'))
plant_node = plant_node[plant_node['month'] == this_month]

In [53]:
plant_node_max = plant_node.max()
plant_node_min = plant_node.min()
plant_node_avg = plant_node.mean()

In [54]:
plant_node_avg

surveyalTime    123.027027
dtype: float64

In [55]:
plant_node.describe()

surveyalTime
count    111.000000
mean     123.027027
std       60.404765
min       52.000000
25%       85.500000
50%      104.000000
75%      143.500000
max      394.000000

### Plant Leaf

In [56]:
plant_leaf = pd.DataFrame(allResponses['04. Plant Monitoring']['Plant Leaf'])
plant_leaf['createdAt'] = plant_leaf['createdAt'].astype('datetime64[ns]')
plant_leaf['month'] = plant_leaf['createdAt'].apply(lambda x : x.strftime('%B'))
plant_leaf = plant_leaf[plant_leaf['month'] == this_month]
plant_leaf_max = plant_leaf.max()
plant_leaf_min = plant_leaf.min()
plant_leaf_avg = plant_leaf.mean()

In [57]:
plant_leaf

deviceIdentifier dataPointId        submissionDate  \
31       PT CAN soil   245520001  2019-10-01T02:52:34Z   
32       PT CAN soil   245520001  2019-10-01T04:02:29Z   
33       PT CAN soil   245520001  2019-10-01T07:12:10Z   
34       PT CAN soil   245520001  2019-10-01T07:27:59Z   
35       PT CAN soil   245520001  2019-10-01T08:54:47Z   
..               ...         ...                   ...   
617      PT CAN soil   245520001  2019-10-01T03:46:10Z   
618      PT CAN soil   245520001  2019-10-01T06:54:16Z   
619      PT CAN soil   245520001  2019-10-03T02:06:16Z   
620      PT CAN soil   245520001  2019-10-02T01:43:54Z   
621      PT CAN soil   245520001  2019-10-03T03:03:16Z   

                   modifiedAt         id      identifier  \
31   2019-10-01T06:01:01.485Z  214470004  23jy-48c7-2pax   
32   2019-10-01T06:02:19.393Z  214470028  23jy-48c7-2pax   
33   2019-10-01T07:18:22.688Z  214470036  23jy-48c7-2pax   
34   2019-10-01T07:28:38.425Z  214470038  23jy-48c7-2pax   
35   2019-10-01T09:00:34.348Z  214490006  23jy-48c7-2pax   
..                        ...        ...             ...   
617  2019-10-01T06:01:43.535Z  252410005  23jy-48c7-2pax   
618  2019-10-01T07:17:47.292Z  252410014  23jy-48c7-2pax   
619  2019-10-03T02:49:26.109Z  252440042  23jy-48c7-2pax   
620  2019-10-02T08:30:44.789Z  252470004  23jy-48c7-2pax   
621  2019-10-15T02:57:45.443Z  254800006  23jy-48c7-2pax   

                                           displayName     formId  \
31   Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
32   Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
33   Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
34   Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
35   Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
..                                                 ...        ...   
617  Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
618  Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
619  Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
620  Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   
621  Demo Plot Bangka - White Pepper - Kepulauan Ba...  247050331   

     surveyalTime    submitter               createdAt    month  
31            295  PT CAN soil 2019-10-01 06:00:52.379  October  
32            268  PT CAN soil 2019-10-01 06:02:14.745  October  
33            140  PT CAN soil 2019-10-01 07:18:16.877  October  
34             89  PT CAN soil 2019-10-01 07:28:33.317  October  
35            163  PT CAN soil 2019-10-01 09:00:27.675  October  
..            ...          ...                     ...      ...  
617           315  PT CAN soil 2019-10-01 06:01:32.387  October  
618           109  PT CAN soil 2019-10-01 07:17:41.287  October  
619           171  PT CAN soil 2019-10-03 02:49:20.037  October  
620           110  PT CAN soil 2019-10-02 08:30:36.304  October  
621            81  PT CAN soil 2019-10-15 02:57:42.167  October  

[111 rows x 12 columns]

### Plant Branch

In [58]:
plant_branch = pd.DataFrame(allResponses['04. Plant Monitoring']['Plant Branch'])
plant_branch['createdAt'] = plant_branch['createdAt'].astype('datetime64[ns]')
plant_branch['month'] = plant_branch['createdAt'].apply(lambda x : x.strftime('%B'))
plant_branch = plant_branch[plant_branch['month'] == this_month]
plant_branch_max = plant_branch.max()
plant_branch_min = plant_branch.min()
plant_branch_avg = plant_branch.mean()

### Other Information

In [59]:
plant_oi = pd.DataFrame(allResponses['04. Plant Monitoring']['Other Information'])
plant_oi['createdAt'] = plant_oi['createdAt'].astype('datetime64[ns]')
plant_oi['month'] = plant_oi['createdAt'].apply(lambda x : x.strftime('%B'))
plant_oi = plant_branch[plant_oi['month'] == this_month]
plant_oi_max = plant_oi.max()
plant_oi_min = plant_oi.min()
plant_oi_avg = plant_oi.mean()

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [60]:
plant_height_max

deviceIdentifier                                           PT CAN soil
dataPointId                                                  245520001
submissionDate                                    2019-10-03T03:03:16Z
modifiedAt                                    2019-10-15T02:57:45.443Z
id                                                           254800006
identifier                                              23jy-48c7-2pax
displayName          Demo Plot Bangka - White Pepper - Kepulauan Ba...
formId                                                       247050331
surveyalTime                                                       394
submitter                                                  PT CAN soil
createdAt                                   2019-10-15 02:57:42.167000
Plant ID                                                            P9
Plant Height (cm)                                                  191
month                                                          October
dtype:

In [61]:
allResponses['05. Activity']

{'Activity': [], 'Fertilizing': [], 'Pesticide': [], 'Watering': []}

## SEND EMAIL

In [62]:
fout = "./testing-real.html"

msg = MIMEMultipart('alternative')
msg['Subject'] = 'DO NOT REPLY: Demo Plot Notification from ' + formatDate(week_ago) + ' to ' + formatDate(today_date)
msg['To'] = ', '.join(EMAIL_RECEPIENTS)
msg['From'] = EMAIL_SENDER

print(checkTime(time.time()) + ' SENDING EMAIL')
with open(fout) as fp:
    msg.attach(MIMEText(fp.read(), 'html'))

try:
    with smtplib.SMTP('smtp.gmail.com', 587) as s:
            s.ehlo()
            s.starttls()
            s.ehlo()
            s.login(EMAIL_SENDER, EMAIL_PASSWORD)
            s.send_message(msg)
            s.quit()
    print(checkTime(time.time()) + ' EMAIL SENT')
except:
    print(checkTime(time.time()) + ' UNABLE TO SEND THE EMAIL. ERROR:',sys.exc_info()[0])
    raise

02:14:49 SENDING EMAIL
02:14:53 UNABLE TO SEND THE EMAIL. ERROR: <class 'smtplib.SMTPAuthenticationError'>


SMTPAuthenticationError: (534, b'5.7.14 <https://accounts.google.com/signin/continue?sarp=1&scc=1&plt=AKgnsbui\n5.7.14 xGtz9Fa0qkZlA9XT8bZsu4COq0joDwLn-VPmLfdxuniYulQpenqdt82urweZ-5sHjXXmIW\n5.7.14 hd4Uvz8J2uj6q1a93-x9v4S8j8bFMngBTeGsbjVR_o-TOH5SF0jC8gEr5E7JVA> Please\n5.7.14 log in via your web browser and then try again.\n5.7.14  Learn more at\n5.7.14  https://support.google.com/mail/answer/78754 69sm15372172pgh.47 - gsmtp')